# Analysis Notebook

This notebook demonstrates loading results from `models/` after running `src/train_models.py`.

In [ ]:
import joblib
from pprint import pprint
m = joblib.load('../models/linear_regression.joblib')
print(m.keys())


**Note:** run `python src/train_models.py --data data/student-mat.csv --output models` before executing cells.